# LogisticRegression

In [1]:
import os
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
%matplotlib inline 
import gensim
from gensim import corpora,models,similarities
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import gensim
from sklearn.metrics import classification_report

In [2]:
df=pd.read_csv("Mock_data_combined.csv",encoding="unicode_escape")
df.head()
df1=df[["Code Chunck","Indicator"]]
df1["Indicator"]=df['Indicator'].astype(int)

F:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [3]:
def logisticmodel():
    # df.columns.tolist()
    #Text Normalization
    tokenized_case = df1['Code Chunck'].apply(lambda x: x.split())
    bow_vectorizer = CountVectorizer(max_df =0.90,min_df=2,max_features=1000,stop_words='english')
    bow=bow_vectorizer.fit_transform(df1['Code Chunck'])
#     bow.shape
    tfidf_vectorizer = TfidfVectorizer(max_df =0.90,min_df=2,max_features=1000,stop_words='english')
    tfidf = tfidf_vectorizer.fit_transform(df1['Code Chunck'])
    wordvec_arrays=np.zeros((len(tokenized_case),100))
    for i in range(len(tokenized_case)):
        wordvec_arrays[i,:]=word_vector(tokenized_case[i],100)
        wordvec_df = pd.DataFrame(wordvec_arrays)
    print(wordvec_df.shape)
    return wordvec_df



In [4]:
def word_vector(tokens,size):
    tokenized_case = df1['Code Chunck'].apply(lambda x: x.split())#tokenizing
    model_w2v = gensim.models.Word2Vec(tokenized_case,size=100,window=5,min_count=2,sg=1,hs=0,negative=10,workers=2,seed=34)
    model_w2v.train(tokenized_case,total_examples = len(df1['Code Chunck']),epochs=20)
#     model_w2v.wv.most_similar(positive='assert')
#     len(model_w2v['assert'])
    vec= np.zeros(size).reshape(1,size)
    count=0.
    for word in tokens:
        try:
            vec +=model_w2v[word].reshape((1,size))
            count+=1.
        except KeyError:
            continue
    if count!=0:
        vec/=count
    return vec   

# LogisticRegression

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [6]:
wordvec_df=logisticmodel()
train_df=wordvec_df
x=train_df
target_df=df1["Indicator"]
y=target_df

F:\Anaconda\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


(50, 100)


In [8]:
x_training_data, x_test_data, y_training_data, y_test_data = train_test_split(x, y, test_size = 0.5)
print(x_training_data)
print("-----------------")
print(x_test_data)
# print(y_training_data)
# print(y_test_data)

          0         1         2         3         4         5         6   \
2  -0.160723 -0.187765 -0.001024 -0.064838  0.009269  0.052114 -0.041524   
34 -0.154637 -0.194553 -0.012597 -0.066328  0.006100  0.054438 -0.014154   
18 -0.162668 -0.188431 -0.013953 -0.054505  0.006854  0.043403 -0.028824   
40 -0.163734 -0.182853 -0.005693 -0.057620 -0.001678  0.044589  0.004949   
14 -0.158917 -0.184059 -0.014213 -0.055137  0.006396  0.042319 -0.029948   
28 -0.157175 -0.179108 -0.008549 -0.056865 -0.002930  0.043000 -0.010646   
33 -0.154637 -0.194553 -0.012597 -0.066328  0.006100  0.054438 -0.014154   
48 -0.150637 -0.175013 -0.010964 -0.054813 -0.006758  0.045131  0.004224   
47 -0.164422 -0.193039 -0.006680 -0.060504 -0.010423  0.044304  0.001090   
20 -0.147663 -0.180210 -0.003192 -0.054821 -0.002717  0.048544 -0.006437   
8  -0.138299 -0.160975 -0.004621 -0.047827  0.005883  0.038484 -0.021642   
38 -0.155597 -0.177174 -0.008024 -0.055936 -0.006313  0.044980  0.007553   
49 -0.150637

In [9]:
model_lg = LogisticRegression()
model_lg.fit(x_training_data, y_training_data)
predictions_lg= model_lg.predict(x_test_data)
predictions_lg

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0])

In [10]:
yprd_df=pd.DataFrame(predictions_lg,index=list(x_test_data.index.values))
yprd_df=yprd_df.rename(columns={0:"Logistic_Predicted_Indicator"})
yprd_df

,Logistic_Predicted_Indicator
45,0
23,0
43,0
15,0
9,0
12,0
29,0
19,0
30,0
6,0


In [11]:
# df1=df1.join(yprd_df)
# df1.to_csv("log_prediction.csv",index=False)

final_df=df1.iloc[list(x_test_data.index.values),:]
final_df=pd.concat([final_df,yprd_df],axis=1)
final_df
final_df.to_csv("log_prediction.csv",index=False)

In [12]:
classification_report(y_test_data, predictions_lg)
print(confusion_matrix(y_test_data, predictions_lg))
y_pred = model_lg.predict(x_test_data)
score =accuracy_score(y_test_data,y_pred)
score

[[10  0]
 [15  0]]


F:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
F:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
F:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.4